In [1]:
import os
import cv2
import pydicom
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

In [ ]:
#Define some directory
base_directory = "/home/khanh/khanh/IMAGE_PROCESSING/3Dircadb1"
print(base_directory)

In [3]:
patients = []
def add_patient(patient_id, folder_name, folder_path):
    patient = {
        "id": patient_id,
        "folder": folder_name,
        "path" : folder_path
    }
    patients.append(patient)

def find_patients_in_directory(base_directory):
    for folder_name in os.listdir(base_directory):
        folder_path = os.path.join(base_directory, folder_name)
        if os.path.isdir(folder_path) and folder_name.startswith("3Dircadb1."):
            patient_id = int(folder_name.split('3Dircadb1.')[1])
            add_patient(patient_id,folder_name,folder_path)

    patients.sort(key=lambda x:x['id'])
    
def patient_direct_path(patient_id):
    patient_dir = patients[patient_id-1]['path'] + "/PATIENT_DICOM"
    return patient_dir

In [ ]:
find_patients_in_directory(base_directory)

print("Patient List:")
for patient in patients:
    print(f"ID: {patient['id']}, Folder: {patient['folder']}")

In [5]:
patient_ID = 1 #Input the ID of patient

In [ ]:
masks = []
def add_mask_for_patient(patient_id, folder_name, total_pixels):
    mask = {
        "id" : patient_id,
        "label" : folder_name,
        "total_pixels" : total_pixels
    }
    masks.append(mask)

def masks_of_patient(patient_id):
    patient_dir = patients[patient_id-1]['path']
    patient_dir += "/MASKS_DICOM"
    for folder_name in os.listdir(patient_dir):
        folder_path = os.path.join(patient_dir,folder_name)
        if os.path.isdir(folder_path):
            add_mask_for_patient(patient_id,folder_name ,0)

    masks.sort(key=lambda x:x['label'])
    return patient_dir

patient_mask_directory = masks_of_patient(patient_ID)
print(patient_mask_directory)

In [ ]:
print("Label List:")
for mask in masks:
    print(f"ID: {mask['id']}, Folder: {mask['label']}")

In [8]:
label = "livertumor05" #Input the label

In [9]:
def get_label(label):
    label_direct = ""
    for mask in masks:
        if mask['label'] == label:
            label_direct = patient_mask_directory + "/" + str(label)
    return label_direct

In [10]:
#Function to load DICOM files from a directory
def load_dicom_files(directory):
    dicom_files = [pydicom.dcmread(os.path.join(directory,f)) for f in os.listdir(directory)]
    return dicom_files

In [11]:
#Function to load a DICCOM file and extract the image data 
def load_dicom_image(dicom_file):
    dicom_data = pydicom.dcmread(dicom_file)
    dicom_image = dicom_data.pixel_array
    return dicom_image

In [12]:
#function to apply windowing
def apply_windowing(image,window_center,window_width):
    window_min = window_center - (window_width / 2)
    window_max = window_center + (window_width / 2)
    image = np.clip(image, window_min, window_max)

    return ((image - window_min) / (window_max - window_min) * 255).astype(np.uint8)

# Organ window settings
organ_window_settings = {
    'Liver': (50, 100),
    'Kidney': (40, 100),
    'Bone': (300, 1500),
}

def apply_window_for_organ(dicom_image, organ):
    if organ in organ_window_settings:
        window_center, window_width = organ_window_settings[organ]
    else:
        raise ValueError(f"Unknown organ '{organ}'. Available options are: {list(organ_window_settings.keys())}")
    
    return apply_windowing(dicom_image.pixel_array, window_center, window_width)

In [13]:
# Function to extract white dots using thresholding
def extract_white_dots(image, threshold_value=200):
    _, thresholded_image = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)
    kernel = np.ones((3, 3), np.uint8)
    cleaned_image = cv2.morphologyEx(thresholded_image, cv2.MORPH_OPEN, kernel)
    return cleaned_image

# Function to overlay white dots onto the windowed patient image in blue
def overlay_white_dots_on_image(base_image, dots_image):
    base_image_rgb = cv2.cvtColor(base_image, cv2.COLOR_GRAY2RGB)
    r, g, b = cv2.split(base_image_rgb)
    b[dots_image == 255] = 255  # Set blue channel to 255 where the dots are
    r[dots_image == 255] = 0    # Set red channel to 0 to prevent yellow
    g[dots_image == 255] = 0    # Set green channel to 0 to prevent yellow
    overlay_image = cv2.merge([r, g, b])
    return overlay_image

In [14]:
def plot_patient_label_mask(patient_file, mask_file, organ):
    # Load the DICOM image for the patient
    dicom_image = pydicom.dcmread(patient_file)
    patient_file_number = os.path.basename(patient_file).split('_')[1]

    # Apply windowing to the patient image based on the specified organ
    patient_image = apply_window_for_organ(dicom_image, organ)
    
    # Load the DICOM label mask
    mask_image = load_dicom_image(mask_file)
    mask_file_number = os.path.basename(mask_file).split('_')[1]

    # Extract white dots from the mask image
    white_dots = extract_white_dots(mask_image, threshold_value=200)

    # Overlay the white dots onto the windowed patient image
    overlay_image = overlay_white_dots_on_image(patient_image, white_dots)

    # Create a figure with 3 subplots
    fig, axs = plt.subplots(1, 3, figsize=(12, 3))

    # Plot the patient DICOM image (grayscale)
    axs[0].imshow(patient_image, cmap='gray')
    axs[0].set_title(f'CT-Scan: Window-{organ}: {patient_file_number}')
    axs[0].axis('off')

    # Plot the label mask image (uncolored)
    axs[1].imshow(mask_image, cmap='gray')
    axs[1].set_title(f'{label.title()}: {mask_file_number}')
    axs[1].axis('off')

    # Plot the patient image with blue dots overlay
    axs[2].imshow(patient_image, cmap='gray')  # Background patient image
    axs[2].imshow(overlay_image, alpha=0.6)  # Overlay the dots with transparency
    axs[2].set_title(f'Color Marking: {label.title()}: {patient_file_number}')
    axs[2].axis('off')

    plt.show()

In [ ]:
# Path to directories containing DICOM files
patient_dir = patient_direct_path(patient_ID)
masks_dir = get_label(label)

print(patient_dir)
print(masks_dir)

# Load the patient DICOM file
patient_files = sorted(
    [os.path.join(patient_dir, f) for f in os.listdir(patient_dir) if f.startswith("image_")],
    key=lambda x: int(os.path.basename(x).split('_')[1])  # Sort by numeric part
)
mask_files = sorted(
    [os.path.join(masks_dir, f) for f in os.listdir(masks_dir) if f.startswith("image_")],
    key=lambda x: int(os.path.basename(x).split('_')[1])  # Sort by numeric part
)

# Display the patient DICOM image, label mask, and color overlay
for patient_file, mask_file in zip(patient_files, mask_files):
    plot_patient_label_mask(patient_file, mask_file, 'Liver')

In [26]:
def count_white_dots(image, threshold_value=200):
    # Create a binary mask where white dots are detected
    _, binary_mask = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)

    # Count the number of white pixels in the binary mask
    white_dot_count = np.sum(binary_mask == 255)  # Count pixels where the mask is white
    return white_dot_count

# Function to process all DICOM images in a folder
def count_white_dots_in_folder(folder_path):
    total_white_dots = 0
    dicom_files = [f for f in os.listdir(folder_path)]  # Filter for DICOM files

    for dicom_file in dicom_files:
        dicom_file_path = os.path.join(folder_path, dicom_file)
        # Load the DICOM image
        image = load_dicom_image(dicom_file_path)
        
        # Count the white dots in the image
        num_white_dots = count_white_dots(image)
        total_white_dots += num_white_dots
        
    return total_white_dots

def count_pixel_of_label():
    total_pixels_liver = 0
    for mask in masks:
        label_count = mask['label']
        mask_count_dir = get_label(label_count)
        number_pixel = count_white_dots_in_folder(mask_count_dir)
        mask['total_pixels'] = number_pixel
        print(f'Total pixels of {label_count}: {number_pixel}')
        if label_count == 'liver':
            total_pixels_liver = number_pixel
    
    return total_pixels_liver

def count_tumors_and_size(total_pixels_of_liver):
    count = 0
    for mask in masks:
        label_count = mask['label']
        if label_count.find('tumor') != -1 and mask['total_pixels'] > 0:
            count+=1
            size_of_tumor = float(int(mask['total_pixels'])/total_pixels_of_liver)
            print(f'Size of {label_count}: {size_of_tumor:5f}')

    print(f'Total tumors of patient: {count}')


In [23]:
total_pixels_of_liver = count_pixel_of_label()

Total pixels of artery: 114923
Total pixels of bone: 717393
Total pixels of check: 13327
Total pixels of leftkidney: 280405
Total pixels of leftlung: 581349
Total pixels of liver: 2865131
Total pixels of liverkyst: 108
Total pixels of livertumor01: 162337
Total pixels of livertumor02: 14464
Total pixels of livertumor03: 7192
Total pixels of livertumor04: 4859
Total pixels of livertumor05: 19426
Total pixels of livertumor06: 766
Total pixels of livertumor07: 5224
Total pixels of portalvein: 103533
Total pixels of rightkidney: 215358
Total pixels of rightlung: 680720
Total pixels of skin: 19301754
Total pixels of spleen: 222773
Total pixels of venoussystem: 143180


In [27]:
count_tumors_and_size(total_pixels_of_liver)

Size of livertumor01: 0.056660
Size of livertumor02: 0.005048
Size of livertumor03: 0.002510
Size of livertumor04: 0.001696
Size of livertumor05: 0.006780
Size of livertumor06: 0.000267
Size of livertumor07: 0.001823
Total tumors of patient: 7
